In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, sum as spark_sum, countDistinct, when
import time

# Initialize Spark session with Delta Lake configuration
spark = SparkSession.builder \
    .appName("spotify-datalake-delta") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1024M") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [ ]:
# Load initial data (v1)
playlists_v1_path = '/shared/sampled/playlists_v1.json'
tracks_v1_path = '/shared/sampled/tracks_v1.json'

playlists_v1_df = spark.read.json(playlists_v1_path)
tracks_v1_df = spark.read.json(tracks_v1_path)

# Save initial data in Delta format

playlists_v1_df.write.format("delta").mode("overwrite").save("/delta/bronze/playlists_v1")
tracks_v1_df.write.format("delta").mode("overwrite").save("/delta/bronze/tracks_v1")

In [ ]:
playlists_v2_path = '/shared/sampled/playlists_v2.json'
tracks_v2_path = '/shared/sampled/tracks_v2.json'

playlists_v2_df = spark.read.json(playlists_v2_path)
tracks_v2_df = spark.read.json(tracks_v2_path)

# Merge playlists_v2 into playlists_v1
from delta.tables import DeltaTable

playlists_delta_table = DeltaTable.forPath(spark, "/delta/bronze/playlists_v1")
playlists_delta_table.alias("old").merge(
    playlists_v2_df.alias("new"),
    "old.pid = new.pid"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

# Merge tracks_v2 into tracks_v1
tracks_delta_table = DeltaTable.forPath(spark, "/delta/bronze/tracks_v1")
tracks_delta_table.alias("old").merge(
    tracks_v2_df.alias("new"),
    "old.pid = new.pid AND old.pos = new.pos"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [ ]:
# Update playlist 11992
playlists_delta_table.update(
    condition = "pid = 11992",
    set = {
        "name": lit("GYM WORKOUT"),
        "collaborative": lit(True)
    }
)

In [ ]:
# Load v3 data
playlists_v3_path = '/shared/sampled/playlists_v3.json'
tracks_v3_path = '/shared/sampled/tracks_v3.json'

playlists_v3_df = spark.read.json(playlists_v3_path)
tracks_v3_df = spark.read.json(tracks_v3_path)

# Merge playlists_v3 into playlists_v1
playlists_delta_table.alias("old").merge(
    playlists_v3_df.alias("new"),
    "old.pid = new.pid"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

# Merge tracks_v3 into tracks_v1
tracks_delta_table.alias("old").merge(
    tracks_v3_df.alias("new"),
    "old.pid = new.pid AND old.pos = new.pos"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [ ]:
# Save updated data in Delta format
playlists_delta_table.toDF().write.format("delta").mode("overwrite").save("/delta/silver/playlists")
tracks_delta_table.toDF().write.format("delta").mode("overwrite").save("/delta/silver/tracks")

In [ ]:
import os

# Measure total time for Delta-based implementation
start_time = time.time()

# (Re-run the above steps here)

end_time = time.time()
delta_total_time = end_time - start_time

# Measure storage usage
parquet_storage = sum(os.path.getsize(f) for f in os.scandir('/path/to/parquet') if f.is_file())
delta_storage = sum(os.path.getsize(f) for f in os.scandir('/path/to/delta') if f.is_file())

print(f"Parquet-based implementation time: {parquet_total_time} seconds")
print(f"Delta-based implementation time: {delta_total_time} seconds")
print(f"Parquet storage usage: {parquet_storage} bytes")
print(f"Delta storage usage: {delta_storage} bytes")

A principal diferença entre as implementações das Tarefas 2 e 3 está no formato de dados utilizado. A Tarefa 2 armazena os dados no formato Parquet, enquanto a Tarefa 3 utiliza Delta Lake, que é baseado no Parquet, mas adiciona funcionalidades avançadas. Na Tarefa 2, as operações de mesclagem e atualização exigem lógica mais complexa e menos eficiente, enquanto na Tarefa 3, o Delta Lake oferece suporte nativo para transações ACID, operações de merge e update, simplificando a implementação e melhorando o desempenho.  

Outra diferença relevante está na evolução de esquema e no controle de concorrência. A Tarefa 2 exige um gerenciamento manual dessas questões, enquanto o Delta Lake, na Tarefa 3, lida automaticamente com a evolução do schema e garante a consistência dos dados. Além disso, a Tarefa 2 não possui suporte nativo para operações simultâneas de read e write, enquanto o Delta Lake permite essas operações mantendo a integridade dos dados. Um diferencial importante do Delta Lake é o suporte a "time travel", que possibilita consultar versões anteriores dos dados, recurso ausente na implementação baseada em Parquet da Tarefa 2.  

Em relação ao desempenho e uso de armazenamento, o Delta Lake tende a ser mais eficiente devido às suas operações otimizadas de mesclagem e atualização, além do suporte a transações ACID. Já o Parquet pode demandar mais tempo, pois exige tratamento manual dessas operações. No armazenamento, o Delta Lake pode ocupar um pouco mais de espaço devido aos metadados adicionais para suporte a transações e versionamento, enquanto o Parquet consome menos espaço por não manter essas informações extras. No geral, o Delta Lake se destaca como uma opção mais robusta para pipelines de dados, garantindo melhor desempenho, consistência e facilidade de manutenção.